## Audio Post-Processing Functions

In [ ]:
import tensorflow as tf
import numpy as np
import librosa
from scipy import signal

### Filter Params

In [ ]:
# Bandpass to filter the noise (for Numpy)
nyq = 0.5 * 16000
low_value = 1000
high_value = 7000
low_cutoff = low_value / nyq
high_cutoff = high_value / nyq

### Import IRs

List of Mic IR
1. IR_Crystal.wav
2. IR_Lomo52A5M.wav
3. IR_OktavaMD57.wav
4. IR_AKGD12.wav
5. IR_GaumontKalee.wav
6. IR_STC4035.wav
7. IR_MelodiumRM6.wav

List of Speaker IR
1. IR_ClestionBD300.wav
2. IR_CelestionV30E606.wav
3. IR_JensenCab.wav
4. IR_Unknown.wav

List of Room IR
1. Room_01.wav
2. Room_02.wav
3. Room_03.wav
4. Room_04.wav
5. Room_05.wav
6. Room_06.wav
7. Room_07.wav
8. Room_08.wav
9. Room_09.wav

***Note:*** Reset path to IRs

In [ ]:
# Speaker IR
ir_speaker_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_speaker/IR_ClestionBD300.wav'
ir_speaker, fs_speaker = librosa.load(ir_speaker_dir, sr=16000, mono=True)

# Microphone IR
ir_mic_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_mic/IR_GaumontKalee.wav'
ir_mic, fs_mic = librosa.load(ir_mic_dir, sr=16000, mono=True)

# Room IR
ir_room_dir = '/beegfs/kp2218/test_runs/conv_test/data/audio/ir_room/Room_01.wav'
ir_room, fs_room = librosa.load(ir_room_dir, sr=16000, mono=True)

### Post Processing function (Tensorflow)
##### Input: Audio tensor (input)
##### Output: Processed audio tensor (audio_out)

In [ ]:
def audio_post_processing(input_file):
    
    with tf.name_scope("audio_post_processing") as scope:
        
        # ------- Speaker IR Convolution -------
            
        # Convolve with Speaker IR
        ir_speaker_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Speaker_Tensor")
        ir_speaker_tensor = tf.expand_dims(ir_speaker_tensor, 1)
        ir_speaker_tensor = tf.expand_dims(ir_speaker_tensor, 2)
        
        # Zero pad with speaker IR shape
        pad_len_speaker = int(len(ir_speaker)/2 -1)
        pad_speaker_value = tf.constant([[0,0], [pad_len_speaker,pad_len_speaker+1], [0,0]], name="Speaker_Pad")
        input_file = tf.pad(input_file, pad_speaker_value, "CONSTANT")

        speaker_out = tf.nn.conv1d(input_file, ir_speaker_tensor, 1, padding="VALID", name="Speaker_Out")
        
        # ------- Noise Parameter -------
        
        # Add noise
        s = speaker_out.get_shape().as_list()
        noise_tensor = tf.random.normal([1,65536,1], mean=0, stddev=5e-3, dtype=tf.float32, name="Noise_param")
        
        # ** Add filter **
        
        speaker_out = tf.add(speaker_out, noise_tensor, name="Speaker_plus_Noise")

        # ------- Room IR Convolution -------
        
        # Convolve with Room IR
        ir_room_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Room_Tensor")
        ir_room_tensor = tf.expand_dims(ir_room_tensor, 1)
        ir_room_tensor = tf.expand_dims(ir_room_tensor, 2)
        
        # Zero pad with Room IR shape
        pad_len_room = int(len(ir_room)/2 -1)
        pad_room_value = tf.constant([[0,0], [pad_len_room,pad_len_room+1], [0,0]])
        speaker_out = tf.pad(speaker_out, pad_room_value, "CONSTANT", name="Room_Pad")
        
        room_out = tf.nn.conv1d(speaker_out, ir_room_tensor, 1, padding="VALID", name="Room_Out")

        # ------- Mic IR Convolution -------
        
        # Convolve with Mic IR
        ir_mic_tensor = tf.placeholder(dtype=tf.float32, shape=[None], name="Mic_Tensor")
        ir_mic_tensor = tf.expand_dims(ir_mic_tensor, 1)
        ir_mic_tensor = tf.expand_dims(ir_mic_tensor, 2)
        
        # Zero pad with Mic IR shape
        pad_len_mic = int(len(ir_mic)/2 -1)
        pad_mic_value = tf.constant([[0,0], [pad_len_mic,pad_len_mic+1], [0,0]], name="Mic_Pad")
        room_out = tf.pad(room_out, pad_mic_value, "CONSTANT")
        
        audio_out = tf.nn.conv1d(room_out, ir_mic_tensor, 1, padding="VALID", name="Audio_Out")

        return audio_out

### Post Processing function (Numpy)
##### Input: Audio array (input)
##### Output: Processed audio array (audio_out)

In [ ]:
def audio_post_processing(input_file):

    # ------- Speaker IR Convolution -------
    
    # Zero pad with speaker IR shape
    pad_speaker = np.zeros(ir_speaker.shape)
    pad_len_speaker = int(len(pad_speaker)/2 -1)
    speaker_pad = pad_speaker[:pad_len_speaker]

    # Convolve with Speaker IR
    input_file = np.concatenate((pad_speaker[:pad_len_speaker+1],
                                 input_file,
                                 pad_speaker[:pad_len_speaker]))
    speaker_out = np.convolve(input_file, ir_speaker, mode='valid')

    # ------- Noise Parameter -------

    # Add noise
    noise_param = np.random.normal(0, 5e-3, size=speaker_out.shape)

    b, a = signal.cheby1(15, 4, [low_cutoff, high_cutoff], btype='bandpass')
    noise_param = signal.filtfilt(b, a, noise_param)

    speaker_out += noise_param

    # ------- Room IR Convolution -------

    # Zero pad with room IR shape
    pad_room = np.zeros(ir_room.shape)
    pad_len_room = int(len(pad_room)/2 -1)
    room_pad = pad_room[:pad_len_room]

    # Convolve with Room IR
    speaker_out = np.concatenate((pad_room[:pad_len_room+1],
                                  speaker_out,
                                  pad_room[:pad_len_room]))
    room_out = np.convolve(speaker_out, ir_room, mode='valid')

    # ------- Mic IR Convolution -------

    # Zero pad with mic IR shape
    pad_mic = np.zeros(ir_mic.shape)
    pad_len_mic = int(len(pad_mic)/2 -1)
    mic_pad = pad_room[:pad_len_mic]

    # Convolve with Mic IR
    room_out = np.concatenate((pad_room[:pad_len_mic+1],
                               room_out,
                               pad_room[:pad_len_mic]))
    audio_out = np.convolve(room_out, ir_mic, mode='valid')

    return audio_out